In [1]:
import json
import os
import tempfile
import pandas as pd
from pdf2image import convert_from_path
import urllib.request
import re

In [2]:
data_path = '../adil-crawler/export/peraturan_2021-05-09'
df = pd.read_json(data_path)

In [3]:
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.loads(f.read())

In [4]:
normal_df = pd.json_normalize(data)
normal_df.head()

,id,source,document,document_status,jenis_peraturan,nomor_peraturan,tahun_peraturan,tentang,tgl_ditetapkan,nomor_ln,...,nomor_bn,nomor_tbn,diubah_oleh,dicabut_oleh,mengubah,menafsirkan,peraturan_pelaksana_dari,daerah_id,nomor_ld,nomor_tld
0,11e44c4ed09124909b9c313231363531,https://peraturan.go.id/peraturan/view.html?id...,[https://peraturan.go.id/common/dokumen/ln/201...,[200],Peraturan Presiden,23,2014,PENDIRIAN UNIVERSITAS TIDAR,2014-04-01,63,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11e44c4ea421ce60bfd2313231353337,https://peraturan.go.id/peraturan/view.html?id...,[https://peraturan.go.id/common/dokumen/ln/200...,"[200, 200]",Undang-Undang,15,2009,Pengesahan Protocol Against The Smuggling of M...,2009-03-16,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11e44c4e535c1b70a21e313231333231,https://peraturan.go.id/peraturan/view.html?id...,[https://peraturan.go.id/common/dokumen/ln/200...,"[200, 200]",Undang-Undang,5,2009,Pengesahan United Nations Convention Against T...,2009-01-12,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11e44c4e9a83706087ca313231353231,https://peraturan.go.id/peraturan/view.html?id...,[https://peraturan.go.id/common/dokumen/ln/200...,"[200, 200]",Undang-Undang,14,2009,"Pengesahan Protocol To Prevent, Suppress and P...",2009-03-05,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11e44c4ecfb12b10a863313231363530,https://peraturan.go.id/peraturan/view.html?id...,[https://peraturan.go.id/common/dokumen/ln/201...,[200],Peraturan Presiden,23,2011,RENCANA AKSI NASIONAL HAK ASASI MANUSIA INDONE...,2011-04-11,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Convert List to String
normal_df['document'] = [''.join(map(str, l)) for l in normal_df['document']]
normal_df['document']

0      https://peraturan.go.id/common/dokumen/ln/2014...
1      https://peraturan.go.id/common/dokumen/ln/2009...
2      https://peraturan.go.id/common/dokumen/ln/2009...
3      https://peraturan.go.id/common/dokumen/ln/2009...
4      https://peraturan.go.id/common/dokumen/ln/2011...
                             ...                        
264    https://peraturan.go.id/common/dokumen/ln/2008...
265    https://peraturan.go.id/common/dokumen/bn/2013...
266    https://peraturan.go.id/common/dokumen/bn/2015...
267    https://peraturan.go.id/common/dokumen/bn/2014...
268    https://peraturan.go.id/common/dokumen/ln/2011...
Name: document, Length: 269, dtype: object

In [ ]:
# Download PDF from URL
count = 0
def download_pdf(download_url, filename):
    response = urllib.request.urlopen(download_url)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()

for doc in normal_df['document']:
    first = doc.rfind("/")
    last = doc.rfind(".")
    doc_name = doc[first + 1 : last]
    try:
        download_pdf(doc, doc_name)
    except:
        print("no pdf")
        count += 1

In [6]:
# Convert PDF to JPEG
dir = '..'
for filename in os.listdir(dir):
    if filename.endswith(".pdf"):
        with tempfile.TemporaryDirectory() as path:
            pages = convert_from_path(filename, output_folder=path)
 
        base_filename  =  os.path.splitext(os.path.basename(filename))[0]      
 
        save_dir = '../adil-data-exploration/images'
    
        for i in range(len(pages)):
            page = pages[i]
            page.save(os.path.join(save_dir, base_filename+"_{}".format(str(i)))+ '.jpg', 'JPEG')
    else:
        continue

